# Semantic search with FAISS (PyTorch)

# Libs

In [24]:
# ! pip install datasets evaluate transformers[sentencepiece]

# reference:
# [1] https://stackoverflow.com/questions/58957169/faiss-error-could-not-find-a-version-that-satisfies-the-requirement-faiss-from/58957380
# [1] Self-summary: 
#   1.1 Python version too high (for example: 3.13 has problem with installing faiss)
#   1.2 must state the cuda version explicitly while installing faiss -> (after install torch) check 'nvidia-...' version in  `conda list > requirement.txt` 
# ! pip install faiss-gpu-cu12 # (NO NEED TO INSTALL THIS, WE DO NOT USE GPU)
# ! pip install faiss-cpu 
# ! pip install torch

In [ ]:
# reference: https://stackoverflow.com/questions/20554074/sklearn-omp-error-15-initializing-libiomp5md-dll-but-found-mk2iomp5md-dll-a
# for solving crashing problem: 22:48:44.157 [error] Disposing session as kernel process died ExitCode: 3, 
# Reason: OMP: Error #15: Initializing libomp140.x86_64.dll, but found libiomp5md.dll already initialized
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
import pandas as pd

# Data processing

In [3]:
df= pd.read_csv("./df_combine.csv")
print(f"row counts in df: {len(df)}")

row counts in df: 1848


In [4]:
# df["claims"][0], df["title"][0]

In [5]:
# convert panda data frame to dataset
from datasets import Dataset

c:\Users\20245580\AppData\Local\anaconda3\envs\workspace_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['title', 'description', 'claims', 'ipc'],
    num_rows: 1848
})

In [7]:
# remove column description
columns = dataset.column_names
columns_to_keep = ["title", "claims", "ipc"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
dataset = dataset.remove_columns(columns_to_remove)
dataset


Dataset({
    features: ['title', 'claims', 'ipc'],
    num_rows: 1848
})

> Now that we **have one comment per row**, let’s **create a new claims_length column** that **contains the number of words per claims**:  

In [8]:
description_dataset = dataset.map(
    lambda x: {"claims_length": len(x["claims"].split())}
)

Map: 100%|██████████| 1848/1848 [00:00<00:00, 9043.55 examples/s]


> We can **use this new column to filter out short comments**, which typically **include things like “cc @lewtun” or “Thanks!” that are not relevant** for our search engine. There’s **no precise number to select for the filter**, **but around 15 words** seems like a good start:  

In [9]:
description_dataset = description_dataset.filter(lambda x: x["claims_length"] > 15)
description_dataset

Filter: 100%|██████████| 1848/1848 [00:00<00:00, 80889.49 examples/s]


Dataset({
    features: ['title', 'claims', 'ipc', 'claims_length'],
    num_rows: 1847
})

In [10]:
# description_dataset[0]['claims_length']

In [11]:
max = 0
for i in range(len(description_dataset)):
    length_description_dataset = description_dataset[i]['claims_length']
    if i== 0:
        min = length_description_dataset
        max = length_description_dataset
    else:
        if length_description_dataset <= min:
            min = length_description_dataset
        
        if length_description_dataset >= max:
            max = length_description_dataset
print(f"min length in dataset: {min} words\nmax length in dataset: {max}")

min length in dataset: 27 words
max length in dataset: 9265


Split the long description into small chunks

In [12]:
# def spilt_into_smaller_descriptions(examples):
#     res = []
#     index = 0
#     num_words_per_chunk = 4500
#     total_chunks = examples["claims"].split()
#     total_len = examples["claims_length"]
#     while index < total_len:
#         chunk = ' '.join(total_chunks[index: index+num_words_per_chunk]) 
#                         # the elem with index = index + num_words_per_chunk 
#                         # is excluded
#         res.append(chunk)
#         index = index + num_words_per_chunk
#     last_chunk = ' '.join(total_chunks[index - num_words_per_chunk: total_len])
#     res.append(last_chunk)
#     return {
#         "claims": res
#     }

In [13]:
# 1. For demonstration purpose:
#    extract only a small fraction of the database.
# 2. For demonstrating the ability to add new data to later the embedding database for similar search:
#    two samples are used. First sample = old data, New sample = new data
all_samples = []
all_samples.append(description_dataset.select(range(20))) # extract row from 0th to 19th
all_samples.append(description_dataset.select(range(20, 40)))

In [14]:
# all_samples[1][0]['title'], all_samples[0][0]['title'] 

In [15]:
# all_samples_split = []
# for idx, elem in enumerate(all_samples):
#     all_samples_split.append(elem.map(spilt_into_smaller_descriptions))

In [16]:
# all_samples_split[1][0]

convert to dataframe to use `explode`

In [17]:
# df_all_samples_split = []
# for idx, elem in enumerate(all_samples_split):
#     elem.set_format("pandas")
#     df_elem = elem[:] # convert to dataframe
#     df_all_samples_split.append(df_elem.explode("claims", ignore_index=True)) # explode


In [18]:
# df_all_samples_split[0].head(4)

Convert the dataframe back to dataset 

In [19]:
# for idx, elem in enumerate(df_all_samples_split):
#     all_samples[idx] =  Dataset.from_pandas(elem)
# print(all_samples)

In [20]:
# all_samples[0][0]['title'],all_samples[1][0]['title'] 

# Creating text embeddings

In [21]:
# import torch
# x = torch.rand(5, 3)
# print(x)

In [ ]:
from transformers import AutoTokenizer, AutoModel
# Because Arnab did not save the tokenizer of the fine-tune model,
# so we should use the tokenizer downloaded from Hugging face of the model Arnab used in his training.
# NOTE: actually, we could combine the model with tokenizer of other model such as:
# token_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1".
# However, not so sure there will be mismatching between the tokenizer id of other model with the used model?
# and how it affects the output?
# But using Arnab fine-tuned model gives much better relevancy test, 
# than using random model like "sentence-transformers/multi-qa-mpnet-base-dot-v1"
token_ckpt = "sadickam/sdg-classification-bert"
model_ckpt = "./current_batch" 
             # this is possible because Arnab saved the model using Huggingface trainer.save_model()
tokenizer = AutoTokenizer.from_pretrained(token_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

c:\Users\20245580\AppData\Local\anaconda3\envs\workspace_1\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\20245580\.cache\huggingface\hub\models--sadickam--sdg-classification-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [49]:
import torch

device = torch.device("cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

 > As we mentioned earlier, we’d **like to represent each entry in our GitHub issues corpus as a single vector**, so we **need to “pool” or average our token embeddings** in some way. One popular approach is to **perform CLS pooling on our model’s outputs**, where we **simply collect the last hidden state for the special [CLS] token**. The following function does the trick for us:

In [24]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

> Next, we’ll create a helper function that will tokenize a list of documents, place the tensors on the GPU, feed them to the model, and finally apply CLS pooling to the outputs:

In [25]:
def get_embeddings(text_list):
    # encoded_input = tokenizer(
    #     text_list, padding=True, truncation=True, return_tensors="pt"
    # )
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [40]:
embedding = get_embeddings(all_samples[0]["claims"][0])
embedding.shape

torch.Size([1, 768])

In [50]:
embedding_all_samples = []
for idx, elem in enumerate(all_samples):
    embedding_all_samples.append( elem.map(
                                    lambda x: {"embeddings": get_embeddings(x["claims"]).detach().cpu().numpy()[0]}
                                    )
                                )   

Map: 100%|██████████| 20/20 [00:07<00:00,  2.83 examples/s]


In [51]:
# for demonstration, assume we only have the first sample being tracked by FAISS:
embedding_all_samples[0].add_faiss_index(column="embeddings")

100%|██████████| 1/1 [00:00<?, ?it/s]


Dataset({
    features: ['title', 'claims', 'ipc', 'claims_length', 'embeddings'],
    num_rows: 20
})

In [52]:
# from the embedded samples, get some idea for testing questions:
embedding_all_samples[0]['claims'][0], embedding_all_samples[0]['title'][0]

("A method of assaying nucleic acids in a sample, comprising the steps of: a) adding multiple sets of probes into the sample to form a mixture, each set of probes comprising: i. a first probe having a first portion at least partially complementary to a first region of a target nucleic acid in the sample and a second portion forming a first primer binding site;ii. a second probe having a first portion at least partially complementary to a second region of the target nucleic acid in the sample and a second portion forming a second primer binding site, wherein the 5' end of the first probe is adjacent to the 3' end of the second probe when both probes are hybridized to the target nucleic acid;b) denaturing nucleic acids in the mixture;c) hybridizing the set of probes to the complementary regions of the target nucleic acid;d) performing a ligation reaction with a ligase enzyme on the set of hybridized probes to connect the adjacent 5' end of the first probe and the 3' end of the second pro

In [53]:
# question = "How can I load a dataset offline?"
question = "How to test nucleic acids in a sample"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [54]:
scores, samples = embedding_all_samples[0].get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [56]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [57]:
for _, row in samples_df.iterrows():
    print(f"TITLE: {row.title}")
    print(f"SCORE: {row.scores}")
    print(f"CLAIMS: {row.claims}")
    print("=" * 50)
    print()

TITLE: POLYMER-BIOACTIVE AGENT CONJUGATES
SCORE: 338.6058349609375
CLAIMS: A bioerodible polymer - bioactive agent conjugate comprising as part of its polymer backbone a moiety of general formula (I):  wherein: A and B, which may be the same or different, represent the remainder of the polymer backbone and are (i) attached to the -O-R(ZD)-O- moiety as shown in formula (I) via a bioerodible moiety, and (ii) comprise a poly(urethane-ester) having monomeric units coupled via bioerodible urethane or ester moieties;R represents a linear or branched optionally substituted hydrocarbon;Z is a linking group; andD is a releasable bioactive agent.The bioerodible polymer - bioactive agent conjugate according to claim 1, wherein A and B each comprise a copolymer of polyurethane and polyester.The bioerodible polymer - bioactive agent conjugate according to claim 1 or claim 2, which comprises less than 25 mol% of polymerised residues that are derived from a C2 diol, relative to the total number of mo

# Save and reload FAISS database

**references:**

[1] https://huggingface.co/docs/datasets/v1.2.0/faiss_and_ea.html

[2] https://discuss.huggingface.co/t/save-and-load-datasets/9260

## Save

IMPORTANT:

[1] must save the dataset which contains the corresponding computed embeddings

In [103]:
for idx, elem in enumerate(embedding_all_samples):
    saved_path = './data_embeddings/epo_dataset' + str(idx)
    print(saved_path)
    try:
        elem.save_to_disk(saved_path)
    except ValueError:
        elem.drop_index('embeddings')
        elem.save_to_disk(saved_path)
# embedding_all_samples[0].save_to_disk('./data_embeddings/epo_dataset0')

./data_embeddings/epo_dataset0


Saving the dataset (1/1 shards): 100%|██████████| 20/20 [00:00<00:00, 3417.64 examples/s]


./data_embeddings/epo_dataset1


Saving the dataset (1/1 shards): 100%|██████████| 20/20 [00:00<00:00, 4125.82 examples/s]


# Searching in multiple databse using FAISS index

[1] https://huggingface.co/learn/cookbook/en/semantic_cache_chroma_vector_database

[2] https://www.pinecone.io/learn/series/faiss/faiss-tutorial/

In [105]:
from datasets import load_from_disk
load_all_dataset = []
for i in range(2):
    dataset_name = './data_embeddings/epo_dataset' + str(i)
    loaded_dataset = load_from_disk(dataset_name)
    load_all_dataset.append(loaded_dataset)
print( load_all_dataset[0],
     len(load_all_dataset)
    )

Dataset({
    features: ['title', 'claims', 'ipc', 'claims_length', 'embeddings'],
    num_rows: 20
}) 2


In [107]:
print(
    load_all_dataset[1],
    load_all_dataset[1]['title'][0],
    load_all_dataset[0]['title'][0]
)

Dataset({
    features: ['title', 'claims', 'ipc', 'claims_length', 'embeddings'],
    num_rows: 20
}) INHALATION PARTICLES: method of preparation METHOD FOR MULTIPLEX NUCLEIC ACID ANALYSIS


In [108]:
from datasets import concatenate_datasets
all_dataset = concatenate_datasets(load_all_dataset)

In [109]:
all_dataset

Dataset({
    features: ['title', 'claims', 'ipc', 'claims_length', 'embeddings'],
    num_rows: 40
})

In [110]:
all_dataset.add_faiss_index(column="embeddings")

100%|██████████| 1/1 [00:00<00:00, 1376.54it/s]


Dataset({
    features: ['title', 'claims', 'ipc', 'claims_length', 'embeddings'],
    num_rows: 40
})

In [111]:
question = "How to test nucleic acids in a sample"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [113]:

scores, samples = all_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

for _, row in samples_df.iterrows():
    print(f"TITLE: {row.title}")
    print(f"SCORE: {row.scores}")
    print(f"DESCRIPTION: {row.claims}")
    print("=" * 50)
    print()


TITLE: COMPOSITIONS FOR INCREASING POLYPEPTIDE STABILITY AND ACTIVITY, AND RELATED METHODS
SCORE: 48.171875
DESCRIPTION: A fusion polypeptide comprising: i) a peptide tag comprising an amino acid sequence that has at least 70% identity to SEQ ID NO: 1, wherein the identity is determined over the entire length of SEQ ID NO: 1; andii) at least one reverse transcriptase polypeptide, wherein the peptide tag is linked to the at least one reverse transcriptase polypeptide, and the peptide tag stabilizes the at least one reverse transcriptase polypeptide when exposed to temperatures between -20°C and 50°C.A kit comprising: a solution comprising the fusion polypeptide of claim 1;a buffer solution; anda solution comprising 2'-deoxynucleoside 5'-triphosphates.The fusion polypeptide of claim 1 or the kit of claim 2, wherein the peptide tag comprises an amino acid sequence that has at least 90% identity to SEQ ID NO: 1, and wherein the identity is determined over the entire length of SEQ ID NO: 1.